# API Handling

## Checking Datastructure  

In [28]:
import requests
url = f"https://pokeapi.co/api/v2/pokemon?limit=150"
def explore_json(data, level=0):
    indent = " " * level
    if isinstance(data,dict):
        for key, value in data.items():
            print(f"{indent}> {key}: {type(value).__name__}, Level:{level}")
            explore_json(value, level + 1) # iterate through the next nested dictionary
    elif isinstance(data,list):
        print(f"{indent}>  List of {len(data)} items, Level: {level}")
        if data:
            explore_json(data[0], level + 1)
data = requests.get(url).json()
explore_json(data)

print(data["results"][0])


> count: int, Level:0
> next: str, Level:0
> previous: NoneType, Level:0
> results: list, Level:0
 >  List of 150 items, Level: 1
  > name: str, Level:2
  > url: str, Level:2
{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'}


In [16]:
import requests
def get_hidden_ability(pokemon_id: int) -> str | bool:
    """
    Retrieves the hidden ability of a Pokémon using the PokéAPI.
    
    Args:
    pokemon_id (int): The Pokémon's ID.
    
    Returns:
    str | bool: The name of the hidden ability, False if none exists, 
                or "Invalid Pokémon ID" if the request fails.
    """
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}"
    response = requests.get(url)
    if response.status_code != 200:
        return "Invalid Pokémon ID"
    data =response.json()
    # print(data)
    for ability in data["abilities"]:
        # print (ability)
        if ability["is_hidden"]:
            # print("found the hidden ability")
            print(f"{data['name']},{ability['ability']['name']}")
            # return the above instead when doing leetcode


# This should be run for you in Leetcode
get_hidden_ability(6)




    # try:
    #     response = requests.get(url)
    #     if response.status_code != 200:
    #         return "Invalid Pokémon ID"
        
    #     data = response.json()
        
    #     # Extract abilities and check for hidden ability
    #     for ability in data["abilities"]:
    #         if ability["is_hidden"]:
    #             return ability["ability"]["name"]
        
    #     return False  # No hidden ability found
    
    # except requests.exceptions.RequestException:
    #     return "Invalid Pokémon ID"

# Test cases
test_cases = [6, 25, 9999]
# for pokemon_id in test_cases:
    # print(f"Pokemon ID {pokemon_id}: {get_hidden_ability(pokemon_id)}")

charizard,solar-power


## String Cleaning


In [45]:
from collections import Counter
sentences = ["the quick brown fox jumps over the wall", 
          "too many cooks spoil the broth", 
          "Games@pi is the greatest!"]
def most_frequent_word_count(sentences):
    """
    Given a list of sentences, find the highest number of occurrences of any word across all sentences.
    Only words consisting of alphabetic characters are considered.
    
    :param sentences: List of sentences (strings)
    :return: The maximum frequency of any single word
    """
    word_count = Counter()
    if sentences:
        for sentence in sentences: # extracts sentences from the list
            for words in sentence.lower().split(): # extracts words from the sentence
                if words.isalpha():
                    word_count[words] += 1 # counts the number of words
        
        print(word_count.most_common(1)[0][0]) 
        #most_common returns a list of the n most common elements
        #-and their counts from the most common to the least.(1) returns the most common word and its count.
        # [0][0] returns the word only
        return word_count.most_common(1)[0][1]
    else:
        return 0
most_frequent_word_count(sentences)


the


4

In [41]:
# Answer Key
word_counts = Counter()

for sentence in sentences:
    words = sentence.lower().split()  # Split sentence into words, convert to lowercase
    # Filter only alphabetic words (ignoring punctuation)
    words = [word for word in words if word.isalpha()]
    word_counts.update(words)  # Update frequency count

print(max(word_counts.values(), default=0))  # Return the max frequency or 0 if empty

# Driver function
def test_most_frequent_word_count():
    test_cases = [
        (["the quick brown fox jumps over the wall", 
          "too many cooks spoil the broth", 
          "Games@pi is the greatest!"], 4),  # "the" appears twice
        
        (["apple banana apple grape", 
          "banana orange banana apple", 
          "grape grape banana"], 4),  # "banana" appears 3 times

        (["Hello world!", 
          "Hello everyone.", 
          "Say hello to the world."], 3),  # "hello" appears 3 times (case insensitive)

        (["A b c d", 
          "B c d e", 
          "C d e f"], 3),  # "c" and "d" appear 3 times each

        (["repeat repeat repeat repeat"], 4),  # Single word repeated

        (["one sentence only"], 1),  # Each word appears once

        ([], 0)  # Empty input
    ]
    
    for i, (sentences, expected) in enumerate(test_cases):
        result = most_frequent_word_count(sentences)
        print(f"Test Case {i+1}: {'PASSED' if result == expected else 'FAILED'} (Expected: {expected}, Got: {result})")

# Run the test cases
test_most_frequent_word_count()

4
the
Test Case 1: PASSED (Expected: 4, Got: 4)
banana
Test Case 2: PASSED (Expected: 4, Got: 4)
hello
Test Case 3: PASSED (Expected: 3, Got: 3)
c
Test Case 4: PASSED (Expected: 3, Got: 3)
repeat
Test Case 5: PASSED (Expected: 4, Got: 4)
one
Test Case 6: PASSED (Expected: 1, Got: 1)
Test Case 7: PASSED (Expected: 0, Got: 0)


# Time Series Anomaly Detection

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Function to detect anomalies based on sliding window average
def detect_anomalies(time_series, window_size=7, threshold=2):
    """
    Detect anomalies in a time series using a sliding window approach.
    
    :param time_series: 1D array of time series data
    :param window_size: size of the sliding window
    :param threshold: the factor by which the current point must deviate from the window average to be considered an anomaly
    :return: List of indices where anomalies were detected
    """
    #Preliminary check for stationarity
    result = adfuller(time_series)
    print(result[1])
    if result[1] > 0.05:
        print("The time series is not stationary")
        return
    else:
        print("The time series is stationary")

    anomalies = []
    half_window = window_size//2
    global_mean = np.mean(time_series)
    c=0
    # We set index range to be from half_window to len(time_series) - half_window to prevent index out of bounds
    # print(range(half_window, len(time_series) - half_window))
    for i in range(half_window, len(time_series) - half_window):
        win = time_series[i-half_window:i+half_window+1] # +1 to include the current point
        print(win)
        window_mean = np.mean(win)
        std = np.std(win)
        print(f"Current element: {time_series[i]} , Mean:{window_mean}, std:{std}, z-score: {abs(time_series[i]-window_mean)/std}")
        if abs(time_series[i] - window_mean) > threshold * std:
            anomalies.append(i)
    return anomalies

time_series_1 = np.array([1, 2, 3, 4, 100, 6, 7, 8, 9, 10, 11, 12])
#perform ADF test

anomalies_1 = detect_anomalies(time_series_1)
print("Anomalies detected at indices:", anomalies_1)

0.0
The time series is stationary
[  1   2   3   4 100   6   7]
Current element: 4 , Mean:17.571428571428573, std:33.708231902002574, z-score: 0.40261466726833295
[  2   3   4 100   6   7   8]
Current element: 100 , Mean:18.571428571428573, std:33.30318364385493, z-score: 2.44506868470506
[  3   4 100   6   7   8   9]
Current element: 6 , Mean:19.571428571428573, std:32.893147974516964, z-score: 0.4125913573836914
[  4 100   6   7   8   9  10]
Current element: 7 , Mean:20.571428571428573, std:32.47793599554338, z-score: 0.4178661037231812
[100   6   7   8   9  10  11]
Current element: 8 , Mean:21.571428571428573, std:32.05734657434634, z-score: 0.4233484683441957
[ 6  7  8  9 10 11 12]
Current element: 9 , Mean:9.0, std:2.0, z-score: 0.0
Anomalies detected at indices: [4]


## Answer Key

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to detect anomalies based on sliding window average
# def detect_anomalies(time_series, window_size=5, threshold=2.0):
#     """
#     Detect anomalies in a time series using a sliding window approach.
    
#     :param time_series: 1D array of time series data
#     :param window_size: size of the sliding window
#     :param threshold: the factor by which the current point must deviate from the window average to be considered an anomaly
#     :return: List of indices where anomalies were detected
#     """
#     anomalies = []
#     half_window = window_size // 2
    
#     for i in range(half_window, len(time_series) - half_window):
#         # Define the window
#         window = time_series[i - half_window:i + half_window + 1]
        
#         # Calculate the mean of the window
#         window_mean = np.mean(window)
        
#         # Compare current point with the window mean
#         if abs(time_series[i] - window_mean) > threshold * np.std(window):
#             anomalies.append(i)
    
#     return anomalies

# # Example test case 1
# time_series_1 = np.array([1, 2, 3, 4, 100, 6, 7, 8, 9, 10, 11, 12])
# anomalies_1 = detect_anomalies(time_series_1)
# print("Anomalies detected at indices:", anomalies_1)

# # Example test case 2
# time_series_2 = np.array([5, 5, 5, 5, 5, 5, 5, 10, 5, 5, 5, 5])
# anomalies_2 = detect_anomalies(time_series_2)
# print("Anomalies detected at indices:", anomalies_2)

# # Example test case 3
# time_series_3 = np.random.normal(0, 1, 50)  # Random data
# time_series_3[10] = 15  # Add a spike
# time_series_3[25] = -10  # Add another spike
# anomalies_3 = detect_anomalies(time_series_3)
# print("Anomalies detected at indices:", anomalies_3)

# # Plot the time series with anomalies highlighted
# def plot_anomalies(time_series, anomalies):
#     plt.figure(figsize=(10, 6))
#     plt.plot(time_series, label="Time Series")
#     plt.scatter(anomalies, time_series[anomalies], color='red', label="Anomalies", zorder=5)
#     plt.legend()
#     plt.show()

# # Plot anomalies for test case 1
# plot_anomalies(time_series_1, anomalies_1)


# 3 Pointer (3SumClosest)

In [ ]:
from typing import List

def threeSumClosest(nums: List[int], target: int) -> int:
    nums=sorted(nums)                           # sort so larger numbers are at the end
    diff=float('inf')                           # initialize diff to infinity
    for i in range(len(nums)-1):                # loop through the sorted list
        start=i+1                               # 1st Pointer starts on the 2nd element
        end=len(nums)-1                         # 3rd Pointer starts on the last element
        while(start<end):                       # loop through the list until start and end meet
            sum=nums[i]+nums[start]+nums[end]   # sum of the 3 numbers
            if sum==target:                     # special case where sum is equal to target
                return target
            elif abs(target-sum)<diff:          # checks if our current sum is closer than the previous one
                diff=abs(target-sum)            # update diff to the new closest sum
                ans=sum                         # update ans to the new closest sum
            if sum>target:                      # Too large, move the end pointer left
                end-=1
            else:
                start+=1                        # Too small, move the start pointer right
    return ans
nums = [-1,2,1,-4]
target = 1
print(threeSumClosest(nums, target)) # 2

2


# 4 Sum

In [ ]:
class Solution:
    def fourSum(self, nums: List[int], target: int) -> List[List[int]]:
        nums.sort()  # Sort the array to enable two pointers technique and easy duplicate skipping
        n = len(nums)
        ans = []  # Final list to collect all unique quadruplets
        
        i = 0
        while i < n:
            j = i + 1
            while j < n:
                l, r = j + 1, n - 1  # Initialize two pointers after j
                goal = target - nums[i] - nums[j]  # Remaining sum needed from nums[l] + nums[r]
                
                while l < r:
                    current_sum = nums[l] + nums[r]
                    if current_sum == goal:
                        ans.append([nums[i], nums[j], nums[l], nums[r]])
                        
                        # Skip duplicates for the third number (nums[l])
                        while l+1 < n and nums[l+1] == nums[l]: 
                            l += 1
                        l += 1  # Move l to the next distinct element
                        r -= 1  # Always move r after a valid quadruplet
                        
                    elif current_sum > goal:
                        r -= 1  # If sum is too big, decrease it by moving right pointer left
                    else:
                        l += 1  # If sum is too small, increase it by moving left pointer right
                
                # After finishing all (l, r) for a fixed (i, j), skip duplicates for the second number (nums[j])
                while j+1 < n and nums[j+1] == nums[j]: 
                    j += 1
                j += 1  # Move to the next distinct j
            
            # After finishing all (j, l, r) for a fixed i, skip duplicates for the first number (nums[i])
            while i+1 < n and nums[i+1] == nums[i]: 
                i += 1
            i += 1  # Move to the next distinct i
        
        return ans  # Return all unique quadruplets


# Top K Elements

In [ ]:
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        
        count = {}  # Dictionary to count the frequency of each element in nums
        
        # Step 1: Count occurrences of each element
        for i in nums:
            count[i] = 1 + count.get(i, 0)  # Increment count if exists, otherwise initialize to 1
        
        res = []  # Final list to store the k most frequent elements
        l = []    # Temporary list to store (count, element) pairs
        
        # Step 2: Build a list of (frequency, element) pairs
        for n, c in count.items():
            l.append((c, n))  # Store count first to enable sorting by frequency
        
        l.sort()  # Step 3: Sort by frequency in ascending order (lowest to highest)
        
        # Step 4: Pop elements with highest frequency until we have k elements
        while len(res) < k:
            res.append(l.pop()[1])  # Pop from the end (highest frequency) and append the element (not the count)
        
        return res  # Step 5: Return the list of top k frequent elements


# Sort Characters by Frequency

In [ ]:
import collections
class Solution:
    """
    This class contains a method to sort characters in a string based on their frequency.
    """
    def frequencySort(self, s: str) -> str:
        # Step 1: Count occurrences manually
        cnt = collections.defaultdict(int)
        for c in s:
            cnt[c] += 1
        # Now, cnt maps each character to its frequency
        # Example: "tree" -> {'t': 1, 'r': 1, 'e': 2}
        
        res = []  # Step 2: Result list to accumulate repeated characters

        # Step 3: Sort characters by frequency descending
        for k, v in sorted(cnt.items(), key=lambda x: -x[1]):
            res += [k] * v  # Repeat character `k` exactly `v` times

        # Step 4: Join all characters into a final string
        return "".join(res)
